# Gesture Recognition
Build a 3D Conv model that will be able to predict the 5 gestures correctly

In [3]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [4]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [5]:
# train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size =  15 #experiment with the batch size

In [4]:
print((len(train_doc)/batch_size))

44.2


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [6]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    master_list = []
    img_idx = [0,4,8,12,16,20,24,28] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(folder_list)/batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),120,120,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    dim = (120,120)
#                     resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
                    resized = imresize(image, (120, 120), interp = 'bicubic')
                    resized_norm = (resized - resized.min())/(resized.max()-resized.min())
                    batch_data[folder,idx,:,:,0] = resized_norm[:,:,0] #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = resized_norm[:,:,1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = resized_norm[:,:,2] #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        rem_batch = (len(folder_list) - (num_batches * batch_size))
        if(len(folder_list) % batch_size) != 0:
            batch_data = np.zeros((batch_size, len(img_idx), 120, 120, 3))
            batch_labels = np.zeros((batch_size, 5))
            for rem_imgs in range(rem_batch):
                folder_img = os.listdir(source_path+'/'+t[rem_imgs + (num_batches * batch_size)].split(';')[0])
                for idx, item in enumerate(img_idx):
                    image = imread(source_path+'/' + t[rem_imgs +  (num_batches * batch_size)].strip().split(';')[0]+'/'+ folder_img[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    dim = (120,120)
#                     resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
                    resized = imresize(image, (120, 120), interp = 'bicubic')
                    resized_norm = (resized - resized.min())/(resized.max()-resized.min())
                    batch_data[folder,idx,:,:,0] = resized_norm[:,:,0] #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = resized_norm[:,:,1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = resized_norm[:,:,2] #normalise and feed in the image
                batch_labels[rem_imgs, int(t[rem_imgs + (num_batches * batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [7]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [8]:
#Conv3D Model
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

img_shape = (8,120,120,3)
#write your model here
model = Sequential()
model.add(Conv3D(8, (3, 3, 3), padding='same',activation='relu', input_shape=img_shape))
model.add(BatchNormalization())
model.add(Conv3D(8, kernel_size = (3, 3, 3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size = (1, 2, 2)))
model.add(Dropout(0.25))

model.add(Conv3D(16, (3, 3, 3), padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Conv3D(16, kernel_size = (3, 3, 3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size = (1, 2, 2)))
model.add(Dropout(0.25))

model.add(Conv3D(32, kernel_size = (3, 3, 3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size = (1, 2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(5))
model.add(Activation('softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [9]:
# optimiser = optimizers.SGD(lr=0.01, momentum=0.0, nesterov=False) #write your optimizer
optimiser = optimizers.Adam(lr=0.001)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 8, 120, 120, 8)    656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 120, 120, 8)    32        
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 6, 118, 118, 8)    1736      
_________________________________________________________________
batch_normalization_2 (Batch (None, 6, 118, 118, 8)    32        
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 6, 59, 59, 8)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 59, 59, 8)      0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 6, 59, 59, 16)     3472      
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [10]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [11]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 5, min_lr = 0.01, verbose = 1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [12]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [13]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 15
Source path =  Project_data/train ; batch size = 15
Epoch 1/50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


45/45 [==============================] - 95s 2s/step - loss: 2.1201 - categorical_accuracy: 0.2696 - val_loss: 2.5544 - val_categorical_accuracy: 0.3143

Epoch 00001: saving model to model_init_2020-03-0904_32_45.032168/model-00001-2.12010-0.26963-2.55442-0.31429.h5
Epoch 2/50
45/45 [==============================] - 22s 478ms/step - loss: 1.3603 - categorical_accuracy: 0.3985 - val_loss: 3.3177 - val_categorical_accuracy: 0.2857

Epoch 00002: saving model to model_init_2020-03-0904_32_45.032168/model-00002-1.36027-0.39852-3.31768-0.28571.h5
Epoch 3/50
45/45 [==============================] - 22s 488ms/step - loss: 1.3004 - categorical_accuracy: 0.4281 - val_loss: 2.2492 - val_categorical_accuracy: 0.3905

Epoch 00003: saving model to model_init_2020-03-0904_32_45.032168/model-00003-1.30044-0.42815-2.24922-0.39048.h5
Epoch 4/50
45/45 [==============================] - 22s 486ms/step - loss: 1.1642 - categorical_accuracy: 0.4593 - val_loss: 3.0563 - val_categorical_accuracy: 0.4095

Epo

In [14]:
#Conv2D + RNN Model
from keras.layers.convolutional import Conv2D, MaxPooling2D

model = Sequential()

model.add(TimeDistributed(Conv2D(8, kernel_size = (3, 3), padding = 'same'), input_shape = img_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(TimeDistributed(Conv2D(16, kernel_size = (3, 3))))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))
model.add(Dropout(0.25))

model.add(TimeDistributed(Conv2D(16, kernel_size = (3, 3), padding = 'same')))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))
model.add(Dropout(0.25))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(64)))
model.add(Dropout(0.25))
model.add(GRU(output_dim = 128, return_sequences = True))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(5, activation = 'softmax'))

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(return_sequences=True, units=128)`


In [15]:
optimiser = optimizers.SGD(lr=0.01, momentum=0.0, nesterov=False) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 8, 120, 120, 8)    224       
_________________________________________________________________
activation_3 (Activation)    (None, 8, 120, 120, 8)    0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 8, 120, 120, 8)    32        
_________________________________________________________________
time_distributed_2 (TimeDist (None, 8, 118, 118, 16)   1168      
_________________________________________________________________
activation_4 (Activation)    (None, 8, 118, 118, 16)   0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 8, 118, 118, 16)   64        
_________________________________________________________________
time_distributed_3 (TimeDist (None, 8, 59, 59, 16)     0         
__________

In [16]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [17]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 5, min_lr = 0.001, verbose = 1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [18]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [19]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 15
Source path =  Project_data/train ; batch size = 15
Epoch 1/50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 1/45 [..............................] - ETA: 3:08 - loss: 1.6224 - categorical_accuracy: 0.3333

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


45/45 [==============================] - 24s 530ms/step - loss: 1.5890 - categorical_accuracy: 0.3274 - val_loss: 1.4182 - val_categorical_accuracy: 0.4286

Epoch 00001: saving model to model_init_2020-03-0904_32_45.032168/model-00001-1.58898-0.32741-1.41816-0.42857.h5
Epoch 2/50
45/45 [==============================] - 21s 475ms/step - loss: 1.0518 - categorical_accuracy: 0.5600 - val_loss: 1.4265 - val_categorical_accuracy: 0.4857

Epoch 00002: saving model to model_init_2020-03-0904_32_45.032168/model-00002-1.05184-0.56000-1.42655-0.48571.h5
Epoch 3/50
45/45 [==============================] - 22s 484ms/step - loss: 0.8073 - categorical_accuracy: 0.6741 - val_loss: 1.7650 - val_categorical_accuracy: 0.4286

Epoch 00003: saving model to model_init_2020-03-0904_32_45.032168/model-00003-0.80734-0.67407-1.76500-0.42857.h5
Epoch 4/50
45/45 [==============================] - 22s 494ms/step - loss: 0.5977 - categorical_accuracy: 0.7452 - val_loss: 1.4532 - val_categorical_accuracy: 0.5429

